<a href="https://colab.research.google.com/github/Bryanper24/Project-1-Github-Data-Analysis-with-Pandas/blob/main/Copy_of_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
CS323 - Assignment 1
Author: [Byran Peraza, Mohammad Ali Khan, Wen Fan ]
Date:   [2/24/26]

FSM-based lexer. Uses DFSMs for:
  - Identifiers
  - Integers
  - Reals
All other token types are handled ad-hoc.
"""

import sys
import os

# ─────────────────────────────────────────────
#  Language definition for Rat26S
# ─────────────────────────────────────────────

KEYWORDS = {
    "while", "if", "else", "endif", "return",
    "get", "put", "integer", "real", "boolean",
    "true", "false", "function", "for", "endwhile"
}

OPERATORS = {"==", "!=", "<=", ">=", "<", ">", "=", "+", "-", "*", "/"}

SEPARATORS = {"(", ")", "{", "}", "[", "]", ",", ";", ":"}


# ─────────────────────────────────────────────
#  DFSM: Identifier
#  RE: [a-zA-Z][a-zA-Z0-9_]*
#
#  States:
#    0 = start
#    1 = accepted (saw a letter)
#    -1 = dead
#
#  Transitions:
#    State 0 --letter--> 1
#    State 1 --letter/digit/underscore--> 1
#    anything else --> dead
# ─────────────────────────────────────────────

def dfsm_identifier(s):
    """
    Run the identifier DFSM on string s.
    Returns the length of the longest valid identifier prefix, or 0.
    """
    state = 0
    i = 0
    last_accept = 0  # no accept yet

    while i < len(s):
        ch = s[i]
        if state == 0:
            if ch.isalpha():
                state = 1
            else:
                break  # dead
        elif state == 1:
            if ch.isalpha() or ch.isdigit() or ch == '_':
                state = 1  # stay
            else:
                break  # done
        i += 1
        if state == 1:
            last_accept = i

    return last_accept


# ─────────────────────────────────────────────
#  DFSM: Integer
#  RE: [0-9]+
#
#  States:
#    0 = start
#    1 = accepted (saw a digit)
#    -1 = dead
# ─────────────────────────────────────────────

def dfsm_integer(s):
    """
    Run the integer DFSM on string s.
    Returns the length of the longest valid integer prefix, or 0.
    """
    state = 0
    i = 0
    last_accept = 0

    while i < len(s):
        ch = s[i]
        if state == 0:
            if ch.isdigit():
                state = 1
            else:
                break
        elif state == 1:
            if ch.isdigit():
                state = 1
            else:
                break
        i += 1
        if state == 1:
            last_accept = i

    return last_accept


# ─────────────────────────────────────────────
#  DFSM: Real
#  RE: [0-9]+\.[0-9]+
#
#  States:
#    0 = start
#    1 = digits before dot
#    2 = saw dot  (not yet accepted)
#    3 = digits after dot  (accepted)
#    -1 = dead
# ─────────────────────────────────────────────

def dfsm_real(s):
    """
    Run the real DFSM on string s.
    Returns the length of the longest valid real prefix, or 0.
    """
    state = 0
    i = 0
    last_accept = 0

    while i < len(s):
        ch = s[i]
        if state == 0:
            if ch.isdigit():
                state = 1
            else:
                break
        elif state == 1:
            if ch.isdigit():
                state = 1
            elif ch == '.':
                state = 2
            else:
                break
        elif state == 2:
            if ch.isdigit():
                state = 3
            else:
                break
        elif state == 3:
            if ch.isdigit():
                state = 3
            else:
                break
        i += 1
        if state == 3:
            last_accept = i

    return last_accept


# ─────────────────────────────────────────────
#  Token record
# ─────────────────────────────────────────────

class Token:
    def __init__(self, token_type, lexeme):
        self.token_type = token_type
        self.lexeme = lexeme

    def __repr__(self):
        return f"Token({self.token_type!r}, {self.lexeme!r})"


# ─────────────────────────────────────────────
#  Lexer
# ─────────────────────────────────────────────

class Lexer:
    def __init__(self, source: str):
        self.source = source
        self.pos = 0

    def _skip_whitespace_and_comments(self):
        """Skip spaces, tabs, newlines, and /* ... */ comments."""
        while self.pos < len(self.source):
            # Whitespace
            if self.source[self.pos].isspace():
                self.pos += 1
                continue

            # Block comment  /* ... */
            if self.source[self.pos:self.pos + 2] == "/*":
                end = self.source.find("*/", self.pos + 2)
                if end == -1:
                    raise SyntaxError(f"Unterminated comment at position {self.pos}")
                self.pos = end + 2
                continue

            break  # nothing left to skip

    def next_token(self):
        """Return the next Token, or None at end of input."""
        self._skip_whitespace_and_comments()

        if self.pos >= len(self.source):
            return None

        remaining = self.source[self.pos:]

        # ── Try Real DFSM first (must beat Integer since both start with digits) ──
        real_len = dfsm_real(remaining)
        if real_len > 0:
            lexeme = remaining[:real_len]
            self.pos += real_len
            return Token("real", lexeme)

        # ── Try Integer DFSM ──
        int_len = dfsm_integer(remaining)
        if int_len > 0:
            lexeme = remaining[:int_len]
            self.pos += int_len
            return Token("integer", lexeme)

        # ── Try Identifier / Keyword DFSM ──
        id_len = dfsm_identifier(remaining)
        if id_len > 0:
            lexeme = remaining[:id_len]
            self.pos += id_len
            if lexeme in KEYWORDS:
                return Token("keyword", lexeme)
            return Token("identifier", lexeme)

        # ── Ad-hoc: two-character operators ──
        two = remaining[:2]
        if two in OPERATORS:
            self.pos += 2
            return Token("operator", two)

        # ── Ad-hoc: one-character operators ──
        one = remaining[0]
        if one in OPERATORS:
            self.pos += 1
            return Token("operator", one)

        # ── Ad-hoc: separators ──
        if one in SEPARATORS:
            self.pos += 1
            return Token("separator", one)

        # ── Unknown character ──
        self.pos += 1
        return Token("unknown", one)

    def tokenize(self):
        """Return a list of all tokens in the source."""
        tokens = []
        while True:
            tok = self.next_token()
            if tok is None:
                break
            tokens.append(tok)
        return tokens


# ─────────────────────────────────────────────
#  Main program
# ─────────────────────────────────────────────

def process_file(input_path: str, output_path: str):
    with open(input_path, "r") as f:
        source = f.read()

    lexer = Lexer(source)

    lines = []
    header = f"{'token':<15} {'lexeme'}"
    separator_line = "-" * 35
    lines.append(header)
    lines.append(separator_line)

    while True:
        tok = lexer.next_token()
        if tok is None:
            break
        lines.append(f"{tok.token_type:<15} {tok.lexeme}")

    output = "\n".join(lines)
    print(output)

    with open(output_path, "w") as f:
        f.write(output + "\n")

    print(f"\n[Output written to {output_path}]")


def main():
    if len(sys.argv) < 2 or sys.argv[1].startswith('-'):
        print("Running built-in test cases...")
        run_tests()
    elif sys.argv[1] == "--test":
        run_tests()
    else:
        input_path = sys.argv[1]
        output_path = sys.argv[2] if len(sys.argv) > 2 else "output.txt"
        process_file(input_path, output_path)


# ─────────────────────────────────────────────
#  Built-in test cases
# ─────────────────────────────────────────────

TEST_CASES = {
   "test1.rat": """
/* Compute the average of two real numbers */
function average(a, b) {
    real result;
    result = (a + b) / 2.00;
    return result;
}
""" ,
"test2.rat": """
/* Count down from n to zero */
integer n;
n = 10;
while (n > 0) {
    put(n);
    n = n - 1;
}
endwhile
""",
"test3.rat": """
/* Grade checker: checks score and prints pass or fail */
integer score;
boolean passed;
get(score);
if (score >= 60) {
    passed = true;
} else {
    passed = false;
}
endif
put(passed);
""",
}


def run_tests():
    os.makedirs("test_outputs", exist_ok=True)

    for filename, source in TEST_CASES.items():
        print("=" * 60)
        print(f"TEST CASE: {filename}")
        print(f"Source:\n{source.strip()}\n")

        lexer = Lexer(source)
        tokens = lexer.tokenize()

        print(f"{'token':<15} {'lexeme'}")
        print("-" * 35)
        for tok in tokens:
            print(f"{tok.token_type:<15} {tok.lexeme}")

        # Write to output file
        out_path = os.path.join("test_outputs", filename.replace(".rat", "_output.txt"))
        with open(out_path, "w") as f:
            f.write(f"Source: {filename}\n")
            f.write(f"{'token':<15} {'lexeme'}\n")
            f.write("-" * 35 + "\n")
            for tok in tokens:
                f.write(f"{tok.token_type:<15} {tok.lexeme}\n")

        print(f"\n[Written to {out_path}]")
        print()


if __name__ == "__main__":
    main()


Running built-in test cases...
TEST CASE: test1.rat
Source:
/* Compute the average of two real numbers */
function average(a, b) {
    real result;
    result = (a + b) / 2.00;
    return result;
}

token           lexeme
-----------------------------------
keyword         function
identifier      average
separator       (
identifier      a
separator       ,
identifier      b
separator       )
separator       {
keyword         real
identifier      result
separator       ;
identifier      result
operator        =
separator       (
identifier      a
operator        +
identifier      b
separator       )
operator        /
real            2.00
separator       ;
keyword         return
identifier      result
separator       ;
separator       }

[Written to test_outputs/test1_output.txt]

TEST CASE: test2.rat
Source:
/* Count down from n to zero */
integer n;
n = 10;
while (n > 0) {
    put(n);
    n = n - 1;
}
endwhile

token           lexeme
-----------------------------------
keyword       